# Yandex.Music

# Contents <a id='back'></a>

* [Introducción](#intro)
* [Etapa 1. Descripción de los datos](#data_review)
    * [Conclusions](#data_review_conclusions)
* [Etapa 2. Data preprocessing](#data_preprocessing)
    * [2.1 Estilo del encabezado](#header_style)
    * [2.2 Valores ausentes](#missing_values)
    * [2.3 Duplicados](#duplicates)
    * [2.4 Conclusiones](#data_preprocessing_conclusions)
* [Etapa 3. Prueba de hipótesis](#hypotheses)
    * [3.1 Hipótesis 1: comparar el comportamiento del usuario en las dos ciudades](#activity)
    * [3.2 Hipótesis 2: música al principio y al final de la semana](#week)
    * [3.3 Hipótesis 3: preferencias de género en Springfield y Shelbyville](#genre)
* [Conclusiones](#end)

## Introducción <a id='intro'></a>
Siempre que investiguemos, necesitamos formular hipótesis que después podamos probar. A veces aceptamos estas hipótesis; otras, las rechazamos. Para tomar las decisiones correctas, una empresa debe ser capaz de entender si está haciendo las suposiciones correctas.

En este proyecto, compararás las preferencias musicales de las ciudades de Springfield y Shelbyville. Estudiarás datos reales de Yandex.Music para probar las hipótesis de abajo y comparar el comportamiento del usuario de esas dos ciudades.

### Objetivo:
Prueba tres hipótesis:
1. La actividad de los usuarios difiere según el día de la semana y dependiendo de la ciudad.
2. Los lunes por la mañana, los habitantes de Springfield y Shelbyville escuchan diferentes géneros. Lo mismo ocurre con los viernes por la noche.
3. Los oyentes de Springfield y Shelbyville tienen preferencias distintas. En Springfield prefieren el pop mientras que en Shelbyville hay más aficionados al rap.

### Etapas
Los datos del comportamiento del usuario se almacenan en el archivo `/datasets/music_project_en.csv`. No hay ninguna información sobre la calidad de los datos así que necesitarás examinarlos antes de probar las hipótesis.

Primero, evaluarás la calidad de los datos y verás si los problemas son significativos. Entonces, durante el preprocesamiento de datos, tomarás en cuenta los problemas más críticos.

Tu proyecto consistirá en tres etapas:
 1. Descripción de los datos
 2. Preprocesamiento de datos
 3. Prueba de hipótesis

[Volver a Contenidos](#back)

## Etapa 1. Descripción de los datos <a id='data_review'></a>

In [42]:
import pandas as pd

In [43]:
# leyendo el archivo y almacenándolo en df
df= pd.read_csv('/content/music_project_en.csv')

In [44]:
df.head()

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Shelbyville,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Springfield,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Shelbyville,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Shelbyville,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Springfield,08:34:34,Monday


In [45]:
df.info()
# obteniendo información general sobre los datos en df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63736 non-null  object
 2   artist    57512 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


La tabla contiene siete columnas. Todas almacenan el mismo tipo de datos: objeto.

De acuerdo con la documentación:
- `'userID'` — identificador del usuario
- `'Track'` — título de la pista
- `'artist'` — nombre del artista
- `'genre'` — género
- `'City'` — ciudad del usuario
- `'time'` — el periodo de tiempo exacto en que se reprodujo la pista
- `'Day'` — día de la semana

Podemos ver tres problemas con el estilo en los nombres de las columnas:
1. Algunos nombres están en mayúsculas, otros en minúsculas.
2. Hay algunos espacios en algunos nombres.
3. Hay que cambiar el estilo y aplicarlo a todos los nombres de las columnas.

El número de valores de las columnas es diferente. Esto significa que los datos contienen valores ausentes.


### Conclusiones <a id='data_review_conclusions'></a>

Cada fila de la tabla almacena datos de la pista que fue reproducida. Algunas columnas describen la pista en sí: su título, el artista y el género. El resto transmite la información del usuario: la ciudad de la que viene, el tiempo que ha reproducido la pista.

Está claro que los datos son suficientes para probar la hipótesis. Sin embargo, hay valores ausentes.

Para continuar, necesitamos preprocesar los datos.

## Etapa 2. Preprocesamiento de datos <a id='data_preprocessing'></a>

In [46]:
print(df.columns)
# la lista de los nombres de las columnas en la tabla df

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')


In [47]:
df.columns = ['user_id','track','artist','genre', 'city', 'time', 'day']
# renombra las columnas

In [48]:
df.info()
# comprobando el resultado: la lista de los nombres de las columnas

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  65079 non-null  object
 1   track    63736 non-null  object
 2   artist   57512 non-null  object
 3   genre    63881 non-null  object
 4   city     65079 non-null  object
 5   time     65079 non-null  object
 6   day      65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


In [49]:
df.isna().sum()
# calculando valores ausentes

user_id       0
track      1343
artist     7567
genre      1198
city          0
time          0
day           0
dtype: int64

No todos los valores ausentes afectan a la investigación. Por ejemplo, los valores ausentes en la pista y artista no son cruciales. Simplemente puedes reemplazarlos por marcadores claros.

Pero los valores ausentes en `'genre'` pueden afectar la comparación entre las preferencias musicales de Springfield y Shelbyville. En la vida real, sería útil saber las razones por las cuales hay datos ausentes e intentar recuperarlos. Pero no tenemos esa oportunidad en este proyecto. Así que tendrás que:
* Rellenar esos valores ausentes con marcadores
* Evaluar cuánto podrían afectar los valores ausentes a tus cómputos.

In [50]:
remplazo = ['track','artist','genre']

for column in remplazo:
  df[column]= df[column].fillna(value='unknown')

df.isna().sum()
# recorriendo los nombres de las columnas y reemplazando los valores ausentes con 'unknown'

user_id    0
track      0
artist     0
genre      0
city       0
time       0
day        0
dtype: int64

In [51]:
df.duplicated().sum()
# contando duplicado obvios

3826

In [52]:
df = df.drop_duplicates().reset_index(drop=True)
df.duplicated().sum()
# eliminando duplicados obvios

0

In [53]:
df_sorted= df['genre'].sort_values()
uniq = df_sorted.unique()

print(uniq)
# inspeccionando los nombres de géneros únicos

['acid' 'acoustic' 'action' 'adult' 'africa' 'afrikaans' 'alternative'
 'ambient' 'americana' 'animated' 'anime' 'arabesk' 'arabic' 'arena'
 'argentinetango' 'art' 'audiobook' 'avantgarde' 'axé' 'baile' 'balkan'
 'beats' 'bigroom' 'black' 'bluegrass' 'blues' 'bollywood' 'bossa'
 'brazilian' 'breakbeat' 'breaks' 'broadway' 'cantautori' 'cantopop'
 'canzone' 'caribbean' 'caucasian' 'celtic' 'chamber' 'children' 'chill'
 'chinese' 'choral' 'christian' 'christmas' 'classical' 'classicmetal'
 'club' 'colombian' 'comedy' 'conjazz' 'contemporary' 'country' 'cuban'
 'dance' 'dancehall' 'dancepop' 'dark' 'death' 'deep' 'deutschrock'
 'deutschspr' 'dirty' 'disco' 'dnb' 'documentary' 'downbeat' 'downtempo'
 'drum' 'dub' 'dubstep' 'eastern' 'easy' 'electronic' 'electropop' 'emo'
 'entehno' 'epicmetal' 'estrada' 'ethnic' 'eurofolk' 'european'
 'experimental' 'extrememetal' 'fado' 'film' 'fitness' 'flamenco' 'folk'
 'folklore' 'folkmetal' 'folkrock' 'folktronica' 'forró' 'frankreich'
 'französisch' 

In [54]:
def replace_wrong_genres(df, wrong_genres, correct_genre):
  df['genre'].replace(wrong_genres, correct_genre, inplace=True)

wrong_genres = ['hip', 'hop', 'hip-hop']
correct_genre = 'hiphop'
replace_wrong_genres(df, wrong_genres, correct_genre)

df_sorted = df['genre'].sort_values()

unique_genres = df_sorted.unique()

print(unique_genres)

# función para reemplazar duplicados implícitos

['acid' 'acoustic' 'action' 'adult' 'africa' 'afrikaans' 'alternative'
 'ambient' 'americana' 'animated' 'anime' 'arabesk' 'arabic' 'arena'
 'argentinetango' 'art' 'audiobook' 'avantgarde' 'axé' 'baile' 'balkan'
 'beats' 'bigroom' 'black' 'bluegrass' 'blues' 'bollywood' 'bossa'
 'brazilian' 'breakbeat' 'breaks' 'broadway' 'cantautori' 'cantopop'
 'canzone' 'caribbean' 'caucasian' 'celtic' 'chamber' 'children' 'chill'
 'chinese' 'choral' 'christian' 'christmas' 'classical' 'classicmetal'
 'club' 'colombian' 'comedy' 'conjazz' 'contemporary' 'country' 'cuban'
 'dance' 'dancehall' 'dancepop' 'dark' 'death' 'deep' 'deutschrock'
 'deutschspr' 'dirty' 'disco' 'dnb' 'documentary' 'downbeat' 'downtempo'
 'drum' 'dub' 'dubstep' 'eastern' 'easy' 'electronic' 'electropop' 'emo'
 'entehno' 'epicmetal' 'estrada' 'ethnic' 'eurofolk' 'european'
 'experimental' 'extrememetal' 'fado' 'film' 'fitness' 'flamenco' 'folk'
 'folklore' 'folkmetal' 'folkrock' 'folktronica' 'forró' 'frankreich'
 'französisch' 

### Conclusiones <a id='data_preprocessing_conclusions'></a>
Detectamos tres problemas con los datos:

- Estilos de encabezados incorrectos
- Valores ausentes
- Duplicados obvios e implícitos

Los encabezados han sido eliminados para conseguir que el procesamiento de la tabla sea más sencillo.

Todos los valores ausentes han sido reemplazados por `'unknown'`. Pero todavía tenemos que ver si los valores ausentes en `'genre'` afectan a nuestros cálculos.

La ausencia de duplicados hará que los resultados sean mas precisos y fáciles de entender.

Ahora ya podemos continuar probando las hipótesis.

## Etapa 3. Prueba de hipótesis <a id='hypotheses'></a>

### Hipótesis 1: comparar el comportamiento del usuario en las dos ciudades <a id='activity'></a>

In [55]:
tracks_by_city_total = df.groupby(['city', 'track']).size().unstack(fill_value=0).sum(axis=1).reset_index(name='total')
tracks_by_city_total
# contando las pistas reproducidas en cada ciudad

,city,total
0,Shelbyville,18512
1,Springfield,42741


In [56]:
df.groupby('day')['track'].count()
# calculando las pistas reproducidas en cada uno de los tres días

day
Friday       21840
Monday       21354
Wednesday    18059
Name: track, dtype: int64

In [68]:
def number_tracks(day, city):
  track_list_count = df[(df['day'] == day) & (df['city'] == city)]
  num_tracks = len(track_list_count['user_id'].unique())
  return num_tracks

# la función cuenta las pistas reproducidas en un cierto día y ciudad.
# primero recupera las filas del día deseado de la tabla,
# después filtra las filas de la ciudad deseada del resultado,
# entonces, encuentra el número de valores de 'user_id' en la tabla filtrada,
# y devuelve ese número.

In [58]:
# el número de canciones reproducidas en Shelbyville el lunes
number_tracks('Monday', 'Shelbyville')


4248

In [59]:
# el número de canciones reproducidas en Springfield el miércoles
number_tracks('Wednesday', 'Springfield')

8388

In [60]:
# el número de canciones reproducidas en Shelbyville el viernes
number_tracks('Friday', 'Shelbyville')


4495

In [61]:
data = {'city': ['Springfield', 'Shelbyville'],
        'monday': [11904, 4248],
        'wednesday': [8388, 5228],
        'friday': [12145, 4495]}

tabla = pd.DataFrame(data)

tabla

,city,monday,wednesday,friday
0,Springfield,11904,8388,12145
1,Shelbyville,4248,5228,4495


**Conclusiones**

Los datos revelan las diferencias en el comportamiento de los usuarios:

- En Springfield, el número de canciones reproducidas alcanzan el punto máximo los lunes y viernes mientras que los miércoles hay un descenso de la actividad.
- En Shelbyville, al contario, los usuarios escuchan más música los miércoles. La actividad de los usuarios los lunes y viernes es menor.

Así que la primera hipótesis parece ser correcta.

### Hipótesis 2: música al principio y al final de la semana <a id='week'></a>

De acuerdo con la segunda hipótesis, los lunes por la mañana y los viernes por la noche los ciudadanos de Springfield escuchan géneros que difieren de aquellos que los usuarios de Shelbyville disfrutan.

In [75]:
spr_general = df.query("city == 'Springfield'")
spr_general

# obteniendo la tabla spr_general de las filas de df,
# donde los valores en la columna 'city' es 'Springfield'
# Crear tabla para Springfield

,user_id,track,artist,genre,city,time,day
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Springfield,14:07:09,Friday
4,E2DC1FAE,Soul People,Space Echo,dance,Springfield,08:34:34,Monday
6,4CB90AA5,True,Roman Messer,dance,Springfield,13:00:07,Wednesday
7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Springfield,20:47:49,Wednesday
8,8FA1D3BE,L’estate,Julia Dalia,ruspop,Springfield,09:17:40,Friday
...,...,...,...,...,...,...,...
61247,83A474E7,I Worship Only What You Bleed,The Black Dahlia Murder,extrememetal,Springfield,21:07:12,Monday
61248,729CBB09,My Name,McLean,rnb,Springfield,13:32:28,Wednesday
61250,C5E3A0D5,Jalopiina,unknown,industrial,Springfield,20:09:26,Friday
61251,321D0506,Freight Train,Chas McDevitt,rock,Springfield,21:43:59,Friday


In [76]:
shel_general = df.query("city=='Shelbyville'")
shel_general

# obteniendo shel_general de las filas df,
# donde el valor de la columna 'city' es 'Shelbyville'
# Crear tabla para Shelbyville

,user_id,track,artist,genre,city,time,day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Shelbyville,20:28:33,Wednesday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Shelbyville,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Shelbyville,08:37:09,Monday
5,842029A1,Chains,Obladaet,rusrap,Shelbyville,13:09:41,Friday
9,E772D5C0,Pessimist,unknown,dance,Shelbyville,21:20:49,Wednesday
...,...,...,...,...,...,...,...
61239,D94F810B,Theme from the Walking Dead,Proyecto Halloween,film,Shelbyville,21:14:40,Monday
61240,BC8EC5CF,Red Lips: Gta (Rover Rework),Rover,electronic,Shelbyville,21:06:50,Monday
61241,29E04611,Bre Petrunko,Perunika Trio,world,Shelbyville,13:56:00,Monday
61242,1B91C621,(Hello) Cloud Mountain,sleepmakeswaves,postrock,Shelbyville,09:22:13,Monday


In [77]:
def genre_weekday (df,day, time1, time2):
  # Filtrar filas por día y por horario
  genre_df = df[(df['day'] == day) & (df['time'] >= time1) & (df['time'] < time2)]

  # Agrupar por género y contar el número de ocurrencias
  genre_df_count = genre_df.groupby('genre').size().reset_index(name='count')

  # Ordenar géneros por número de ocurrencias y seleccionar los 15 más populares
  genre_df_sorted = genre_df_count.sort_values(by='count', ascending=False).head(15)

  # Devolver tabla con los 15 géneros más populares
  return genre_df_sorted

# declarando la función genre_weekday() con los parámetros day=, time1= y time2=. Debería
# devolver información sobre los géneros más populares de un determinado día a una determinada hora

In [78]:
result = genre_weekday(spr_general, 'Monday', '00:00:00', '12:00:00')
print(result)
# llamando a la función para el lunes por la mañana en Springfield (utilizando spr_general en vez de la tabla df)

           genre  count
96           pop    781
30         dance    549
42    electronic    480
112         rock    474
63        hiphop    286
114       ruspop    186
146        world    181
115       rusrap    175
2    alternative    164
139      unknown    161
24     classical    157
82         metal    120
71          jazz    100
51          folk     97
124   soundtrack     95


In [79]:
# llamando a la función para el lunes por la mañana en Shelbyville (utilizando shel_general en vez de la tabla df)
result = genre_weekday(shel_general, 'Monday', '00:00:00', '12:00:00')
print(result)

           genre  count
68           pop    218
18         dance    182
81          rock    162
28    electronic    147
45        hiphop     80
83        ruspop     64
1    alternative     58
84        rusrap     55
51          jazz     44
13     classical     40
107        world     36
75           rap     32
90    soundtrack     31
79           rnb     27
60         metal     27


In [80]:
# llamando a la función para el viernes por la tarde en Shelbyville
result = genre_weekday(shel_general, 'Friday', '12:00:00', '18:00:00')
print(result)

           genre  count
68           pop    281
17         dance    237
81          rock    221
25    electronic    173
41        hiphop    123
13     classical    106
0    alternative     83
82        ruspop     76
83        rusrap     59
104        world     57
48          jazz     45
31          folk     42
87    soundtrack     38
80           rnb     34
54         metal     34


**Conclusión**

Habiendo comparado los 15 géneros más populares del lunes por la mañana podemos concluir lo siguiente:

1. Los usuarios de Springfield y Shelbyville escuchan música similar. Los cinco géneros más populares son los mismos, solo rock y electrónica han intercambiado posiciones.

2. En Springfield el número de valores ausentes resultaron ser tan altos que el valor `'unknown'` llegó al décimo. Esto significa que los valores ausentes forman una parte considerable de los datos, lo que podría ser la base de la cuestión sobre la fiabilidad de nuestras conclusiones.

Para el viernes por la tarde, la situación es similar. Los géneros individuales varían algo pero, en general, los 15 más populares son parecidos en las dos ciudades.

De esta forma, la segunda hipótesis ha sido parcialmente demostrada:
* Los usuarios escuchan música similar al principio y al final de la semana.
* No hay una gran diferencia entre Springfield y Shelbyville. En ambas ciudades, el pop es el género más popular.

Sin embargo, el número de valores ausentes hace este resultado un tanto cuestionable. En Springfield, hay tantos que afectan a nuestros 15 más populares. De no faltarnos esos valores, las cosas podrían parecer diferentes.

### Hipótesis 3: preferencias de género en Springfield y Shelbyville <a id='genre'></a>

Hipótesis: Shelbyville ama la música rap. A los ciudadanos de Springfield les gusta más el pop.

In [70]:
# Agrupar DataFrame por género y contar el número de filas por cada género
spr_genres = spr_general.groupby('genre').size().reset_index(name='count')

# Ordenar tabla por número de ocurrencias en orden descendente
spr_genres = spr_genres.sort_values(by='count', ascending=False)

# Mostrar resultado
spr_genres

,genre,count
170,pop,5892
53,dance,4435
194,rock,3965
72,electronic,3786
109,hiphop,2096
...,...,...
146,metalcore,1
141,marschmusik,1
140,malaysian,1
139,lovers,1


In [72]:
print(spr_genres.head(10))

# imprimiendo las 10 primeras filas de spr_genres

           genre  count
170          pop   5892
53         dance   4435
194         rock   3965
72    electronic   3786
109       hiphop   2096
44     classical   1616
247        world   1432
6    alternative   1379
197       ruspop   1372
198       rusrap   1161


In [71]:
shel_genres = shel_general.groupby('genre').size().reset_index(name='count')

# cuenta los valores 'genre' en el agrupamiento con count(),
shel_genres = shel_genres.sort_values(by='count', ascending=False)

# ordena el Series resultante en orden descendente y guárdalo en shel_genres
shel_genres

,genre,count
138,pop,2431
41,dance,1932
159,rock,1879
56,electronic,1736
90,hiphop,960
...,...,...
114,mandopop,1
111,leftfield,1
109,laiko,1
105,jungle,1


In [74]:
# imprimiendo las 10 primeras filas de shel_genres
print(shel_genres.head(10))


           genre  count
138          pop   2431
41         dance   1932
159         rock   1879
56    electronic   1736
90        hiphop    960
3    alternative    649
33     classical    646
163       rusrap    564
162       ruspop    538
200        world    515


**Conclusión**

La hipótesis ha sido parcialmente demostrada:
* La música pop es el género más popular en Springfield, tal como se esperaba.
* Sin embargo, la música pop ha resultado ser igual de popular en Springfield que en Shelbyville y el rap no estaba entre los 5 más populares en ninguna de las ciudades.


# Conclusiones <a id='end'></a>

Hemos probado las siguientes tres hipótesis:

1. La actividad de los usuarios difiere dependiendo del día de la semana y de las distintas ciudades.
2. Los lunes por la mañana los residentes de Springfield y Shelbyville escuchan géneros distintos. Lo mismo ocurre con los viernes por la noche.
3. Los oyentes de Springfield y Shelbyville tienen distintas preferencias. En ambas ciudades, Springfield y Shelbyville, se prefiere el pop.

Tras analizar los datos, concluimos:

1. La actividad del usuario en Springfield y Shelbyville depende del día de la semana aunque las ciudades varían de diferentes formas.

La primera hipótesis ha sido aceptada completamente.

2. Las preferencias musicales no varían significativamente en el transcurso de la semana en Springfield y Shelbyville. Podemos observar pequeñas diferencias en el orden los lunes, pero:
* En Springfield y Shelbyville la gente lo que más escucha es la música pop.

Así que no podemos aceptar esta hipótesis. También debemos tener en cuenta que el resultado podría haber sido diferente si no fuera por los valores ausentes.

3. Resulta que las preferencias musicales de los usuarios de Springfield y Shelbyville son bastante parecidas.

La tercera hipótesis es rechazada. Si hay alguna diferencia en las preferencias no se puede observar en los datos.

### Nota
En los proyectos reales, la investigación supone el estudio de hipótesis estadísticas que es más preciso y cuantitativo. También ten en cuenta que no siempre podemos sacar conclusiones sobre una ciudad entera basándonos en datos de una sola fuente.

Analizarás el estudio de hipótesis en el sprint de análisis estadístico de datos.